In [ ]:
// From day 1
let chunkBy isSeparator items =
    let startNewChunk chunks =
        []::chunks
    
    let addToCurrentChunk (item:'a) (chunks : 'a list list) =
        match chunks with
        | [] -> 
            [[item]]
        | currentChunk::previousChunks ->
            (item::currentChunk)::previousChunks

    let rec processItems chunks remainingItems =
        match remainingItems with
        | [] -> 
            chunks
        | item::rest ->
            let newChunks =
                if isSeparator item then
                    chunks |> startNewChunk
                else
                    chunks |> addToCurrentChunk item
            processItems newChunks rest

    processItems [] items
    |> List.map List.rev
    |> List.rev

In [ ]:
open Checked 

type Operator = | Add | Multiply
type Operand = | Old | Value of int

type MonkeyRules =
    {
        Order : int
        Operator : Operator
        Operand : Operand
        DivisbleBy : int
        ThrowTrue : int
        ThrowFalse : int
    }

type MonkeyState = 
    { 
        Inspections : uint64
        Holding : uint64 list
    }    

// Assumes a very rigid format, no error checking!
let parseMonkey (lines : string[])  =
    //Monkey 0:
    let monkey = 
        lines[0].Split([| ' '; ':' |], StringSplitOptions.RemoveEmptyEntries) 
        |> Array.last
        |> int

    //Starting items: 79, 98
    let items = 
        lines[1].Split([|':';','|], StringSplitOptions.TrimEntries)
        |> Seq.skip 1
        |> Seq.map uint64
        |> List.ofSeq

    //Operation: new = old * 19
    let operator, operand = 
        let op = lines[2].Split("old", 2, StringSplitOptions.TrimEntries)[1]
        let operand = 
            match op.Substring(2) with
            | "old" -> Old
            | x -> Value (int x)
        let operator =
            match op[0] with
            | '+' -> Add
            | '*' -> Multiply
            | x -> failwith $"Unknown operator {x}"
        operator, operand

    //Test: divisible by 23 
    let divisibleBy = 
        lines[3].Split("by", StringSplitOptions.TrimEntries)[1]
        |> int

    //If true: throw to monkey 2
    let trueThrow = 
        lines[4].Split(' ') |> Array.last |> int

    // If false: throw to monkey 3
    let falseThrow =
        lines[5].Split(' ') |> Array.last |> int

    let rules = 
        {
            Order = monkey
            Operator = operator
            Operand = operand
            DivisbleBy = divisibleBy
            ThrowTrue = trueThrow
            ThrowFalse = falseThrow
        }
    rules, items
    
let parseMonkeys lines = 
    let parsed =
        lines 
        |> List.ofSeq
        |> chunkBy String.IsNullOrEmpty
        |> List.map (Array.ofList >> parseMonkey)
    
    let rules =
        parsed
        |> List.map (fun (r,_) -> (r.Order,r))
        |> Map.ofList

    let state =
        parsed
        |> List.map (fun (r,i) -> (r.Order, { Inspections = 0uL; Holding = i}))
        |> Map.ofList

    (rules, state)

let applyOperation (monkey : MonkeyRules) item =
    let value = 
        match monkey.Operand with
        | Old -> item
        | Value v -> uint64 v
    
    match monkey.Operator with
    | Add -> item + value
    | Multiply -> item * value

let applyTest (monkey : MonkeyRules) (item : uint64) =
    if item % (uint64 monkey.DivisbleBy) = 0uL then
        monkey.ThrowTrue
    else monkey.ThrowFalse

let processMonkey modulo (monkey : MonkeyRules) (monkeyStates : Map<int,MonkeyState>) =
    let items  = monkeyStates[monkey.Order]
    items.Holding
    |> List.fold (
        fun state item ->
            //printfn "Applying"
            let worry = applyOperation monkey item
            //printfn "Applied"
            let worry = worry % modulo
            //printfn "Mod"
            let transferTo = applyTest monkey worry
            let destMonkeyState = state |> Map.find transferTo
            let transferred = { destMonkeyState with Holding = worry::(destMonkeyState.Holding) }
            
            Map.add transferTo transferred state 
        ) monkeyStates
    |> Map.add monkey.Order { Inspections = items.Inspections + uint64 items.Holding.Length; Holding = [] }

let performRound monkeyRules monkeyState =
    let modulo = 
        monkeyRules
        |> Map.toSeq
        |> Seq.map (fun (_,m) -> uint64 m.DivisbleBy)
        |> Seq.fold (*) 1uL

    monkeyRules
    |> Map.fold (
        fun state _ monkey ->
            processMonkey modulo monkey state
        ) monkeyState


In [ ]:
#r "nuget: FsUnit"
open FsUnitTyped

let shouldHaveSameElements xs ys =
    xs |> List.sort |> shouldEqual (ys |> List.sort)

let testInput =
    [|
        "Monkey 0:"
        "  Starting items: 79, 98"
        "  Operation: new = old * 19"
        "  Test: divisible by 23"
        "    If true: throw to monkey 2"
        "    If false: throw to monkey 3"
        ""
        "Monkey 1:"
        "  Starting items: 54, 65, 75, 74"
        "  Operation: new = old + 6"
        "  Test: divisible by 19"
        "    If true: throw to monkey 2"
        "    If false: throw to monkey 0"
        ""
        "Monkey 2:"
        "  Starting items: 79, 60, 97"
        "  Operation: new = old * old"
        "  Test: divisible by 13"
        "    If true: throw to monkey 1"
        "    If false: throw to monkey 3"
        ""
        "Monkey 3:"
        "  Starting items: 74"
        "  Operation: new = old + 3"
        "  Test: divisible by 17"
        "    If true: throw to monkey 0"
        "    If false: throw to monkey 1"
    |]

let testMonkeyRules, testMonkeyState =
    parseMonkeys testInput

let afterRounds =
    [|1..20|]
    |> Array.scan (
        fun state _ ->
            performRound testMonkeyRules state
        ) testMonkeyState

afterRounds[1].[0].Holding |> shouldHaveSameElements [ 20; 23; 27; 26 ]
afterRounds[1].[1].Holding |> shouldHaveSameElements [ 2080; 25; 167; 207; 401; 1046 ]
afterRounds[1].[2].Holding |> shouldHaveSameElements []
afterRounds[1].[3].Holding |> shouldHaveSameElements []

afterRounds[2].[0].Holding |> shouldHaveSameElements [ 695; 10; 71; 135; 350; ]
afterRounds[2].[1].Holding |> shouldHaveSameElements [ 43; 49; 58; 55; 362 ]
afterRounds[2].[2].Holding |> shouldHaveSameElements []
afterRounds[2].[3].Holding |> shouldHaveSameElements []

afterRounds[3].[0].Holding |> shouldHaveSameElements [ 16; 18; 21; 20; 122]
afterRounds[3].[1].Holding |> shouldHaveSameElements [ 1468; 22; 150; 286; 739]
afterRounds[3].[2].Holding |> shouldHaveSameElements []
afterRounds[3].[3].Holding |> shouldHaveSameElements []

afterRounds[4].[0].Holding |> shouldHaveSameElements [ 491; 9; 52; 97; 248; 34]
afterRounds[4].[1].Holding |> shouldHaveSameElements [ 39; 45; 43; 258]
afterRounds[4].[2].Holding |> shouldHaveSameElements []
afterRounds[4].[3].Holding |> shouldHaveSameElements []

afterRounds[5].[0].Holding |> shouldHaveSameElements [ 15; 17; 16; 88; 1037 ]
afterRounds[5].[1].Holding |> shouldHaveSameElements [ 20; 110; 205; 524; 72 ]
afterRounds[5].[2].Holding |> shouldHaveSameElements []
afterRounds[5].[3].Holding |> shouldHaveSameElements []

afterRounds[6].[0].Holding |> shouldHaveSameElements [ 8; 70; 176; 26; 34 ]
afterRounds[6].[1].Holding |> shouldHaveSameElements [ 481; 32; 36; 186; 2190 ]
afterRounds[6].[2].Holding |> shouldHaveSameElements []
afterRounds[6].[3].Holding |> shouldHaveSameElements []

afterRounds[7].[0].Holding |> shouldHaveSameElements [ 162; 12; 14; 64; 732; 17]
afterRounds[7].[1].Holding |> shouldHaveSameElements [ 148; 372; 55; 72]
afterRounds[7].[2].Holding |> shouldHaveSameElements []
afterRounds[7].[3].Holding |> shouldHaveSameElements []

afterRounds[8].[0].Holding |> shouldHaveSameElements [ 51; 126; 20; 26; 136]
afterRounds[8].[1].Holding |> shouldHaveSameElements [ 343; 26; 30; 1546; 36]
afterRounds[8].[2].Holding |> shouldHaveSameElements []
afterRounds[8].[3].Holding |> shouldHaveSameElements []

afterRounds[9].[0].Holding |> shouldHaveSameElements [ 116; 10; 12; 517; 14]
afterRounds[9].[1].Holding |> shouldHaveSameElements [ 108; 267; 43; 55; 288]
afterRounds[9].[2].Holding |> shouldHaveSameElements []
afterRounds[9].[3].Holding |> shouldHaveSameElements []

afterRounds[10].[0].Holding |> shouldHaveSameElements [ 91; 16; 20; 98]
afterRounds[10].[1].Holding |> shouldHaveSameElements [ 481; 245; 22; 26; 1092; 30]
afterRounds[10].[2].Holding |> shouldHaveSameElements []
afterRounds[10].[3].Holding |> shouldHaveSameElements []

afterRounds[15].[0].Holding |> shouldHaveSameElements [  83; 44; 8; 184; 9; 20; 26; 102]
afterRounds[15].[1].Holding |> shouldHaveSameElements [  110; 36]
afterRounds[15].[2].Holding |> shouldHaveSameElements []
afterRounds[15].[3].Holding |> shouldHaveSameElements []

afterRounds[20].[0].Holding |> shouldHaveSameElements [ 10; 12; 14; 26; 34]
afterRounds[20].[1].Holding |> shouldHaveSameElements [ 245; 93; 53; 199; 115]
afterRounds[20].[2].Holding |> shouldHaveSameElements []
afterRounds[20].[3].Holding |> shouldHaveSameElements []

afterRounds[20].[0].Inspections |> shouldEqual 101        
afterRounds[20].[1].Inspections |> shouldEqual 95
afterRounds[20].[2].Inspections |> shouldEqual 7
afterRounds[20].[3].Inspections |> shouldEqual 105

In [ ]:
open System.IO

let sourcePath = Path.Combine(__SOURCE_DIRECTORY__, "input_11.txt")
let monkeyRules, monkeyState = 
    File.ReadAllLines(sourcePath)
    |> parseMonkeys

let afterRound20 = 
    [1..20]
    |> List.fold (
        fun state _ ->
            performRound monkeyRules state
        ) monkeyState   

let mostActive =
    afterRound20 
    |> Map.toList
    |> List.map (fun (_,m) -> m.Inspections)
    |> List.sortDescending
    |> List.take 2

let level = 
    match mostActive with
    | [m1;m2] -> m1*m2
    | _ -> failwith "Did not take 2?"             

In [ ]:
printfn "Monkey business is %d" level

## Part 2

In [ ]:
let afterRound10000 = 
    [1..10000]
    |> List.fold (
        fun state _ ->
            performRound monkeyRules state
        ) monkeyState 

In [ ]:
let mostActive =
    afterRound10000 
    |> Map.toList
    |> List.map (fun (_,m) -> m.Inspections)
    |> List.sortDescending
    |> List.take 2

let level = 
    match mostActive with
    | [m1;m2] -> m1*m2
    | _ -> failwith "Did not take 2?" 

In [ ]:
printfn "Level is now %d" level